In [ ]:
from Robinhood import Robinhood
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import datetime as dt

==================================================tickers============================================================

In [ ]:
#technical indicators
#column names: time, open_price, close_price, high_price, low_price, volume

#Average True Range
def atr(df,n):
    df['h-l']=abs(df['high_price']-df['low_price'])
    df['h-pc']=abs(df['high_price']-df['close_price'].shift(1))
    df['l-pc']=abs(df['low_price']-df['close_price'].shift(1))
    df['tr']=df[['h-l','h-pc','l-pc']].max(axis=1,skipna=False)
    df['atr'] = df['tr'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    #df = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df['atr']

#cumulative annual growth rate
def cagr(df):
    #df['ret'] = df['close_price'].pct_change()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    n = len(df)/len(df)
    cagr = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return cagr

#volatility
def volatility(df):
    #df['ret'] = df['close_price'].pct_change()
    vol = df["ret"].std() * np.sqrt(len(df))
    return vol

#sharpe ratio, rf is the risk free ratio

def sharpe(df, rf):
    sr = (cagr(df) - rf)/volatility(df)
    return sr

def max_dd(df):
    df["cum_return"] = (1 + df["ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

In [ ]:
#get historical trade data
def get_ohlcv(robinhood_client, tickers, itvl, prd): #tickers is a list of tickers, itvl: interval, prd: period
    ohlcv = {}
    for ticker in tickers:
        print(ticker)
        print('retreiving data for ' + ticker)
        data = robinhood_client.get_historical_quotes(ticker, itvl, prd)['results'][0]['historicals']
        df = pd.DataFrame(data = data)
#        df = df.dropna(inplace = True)
#        print(df)
        df.columns.values[0] = "time"
        df["time"] = pd.to_datetime(df['time'])  #'%Y-%m-%dT%H:%M:%SZ
        df = df.set_index("time")
        df.index = df.index.tz_convert('US/Eastern')
        #df["time"] = dt.datetime.strptime(df['time'].astype(str), "%Y-%m-%d %H:%M:%S")
        df = df.iloc[:, :-2]
        df[['open_price', 'close_price', 'high_price', 'low_price']] = df[['open_price', 'close_price', 'high_price', 'low_price']].apply(pd.to_numeric)
        #=====================
        df['atr'] = atr(df, 20)
        df['roll_max_cp'] = df['high_price'].rolling(20).max()
        df['roll_min_cp'] = df['low_price'].rolling(20).max()
        df['roll_max_vol'] = df['volume'].rolling(20).max()
        ohlcv[ticker] = df
    return ohlcv

In [ ]:
robinhood_client = Robinhood()
robinhood_client.login(username = 'yiwenluo1@outlook.com', password = 'lyw729rolling')

In [ ]:
#import stock ticers from robinhood lists
def scrape_table(table_url):
    #get table
    page = requests.get(table_url)
    page_content = page.content
    soup = BeautifulSoup(page_content, 'html.parser')
    table = soup.find_all('table', {'id': 'constituents'})

    #read scraped data into list
    all_rows = [] 
    for t in table:
        rows = t.find_all('tr')
        for row in rows:
            value = row.get_text()
            all_rows.append(value)

    #convert list to dataframe
    df = pd.DataFrame(all_rows)
    df = df[0].str.split('\n', expand = True)
    df = df.iloc[1:, [1, 3]]
    df.columns = ['symbol', 'security']
    return df

#append last trade price to tickers, price is converted into float
def append_price(df):
    df['last_trade_price'] = df['symbol'].apply(lambda x: robinhood_client.quote_data(x)['last_trade_price'])
    df['last_trade_price'] = df['last_trade_price'].apply(pd.to_numeric)
    return df


#s&p 500 wikipedia url
snp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

snp = append_price(scrape_table(snp_url))

snp_s = snp[snp.last_trade_price < 50]
my_tickers = snp_s['symbol'].values.tolist()

ohlcv = get_ohlcv(robinhood_client, my_tickers, '5minute', 'week')

In [ ]:
#0810 TOP MOVERS
my_tickers = ['GRPN', 'TDC', 'UFS', 'SG', 'TRUE', 'NUZE', 'RC', 'SSTI', 'OCFT', 'YELP', 'VIVO', 'PETQ', 'PDFS', 'NH']
ohlcv = get_ohlcv(robinhood_client, my_tickers, '5minute', 'week')

In [ ]:
my_tickers = ['FCX', 'CTL', 'BKR', 'RF', 'SYF', 'UNM', 'OKE', 'NLOK', 'IVZ']
ohlcv = get_ohlcv(robinhood_client, my_tickers, '5minute', 'week')

In [ ]:
#cannabis tickers
my_tickers = ['ACB', 'CGC', 'CRON', 'TLRY', 'SSPK', 'HEXO', 'OGI', 'SPRWF', 
              'SNDL', 'CNTTQ', 'HUGE', 'GTEC', 'SCNA', 'TRTC', 'PURA', 'HEMP',
             'HMPQ']
ohlcv = get_ohlcv(robinhood_client, my_tickers, '5minute', 'week')

In [ ]:
#top movers tickers
my_tickers = ['OSTK', 'PRK', 'SBH', 'MTLS', 'UCTT', 'TSE', 'HOME', 'PDFS', 'LMNL', 
              'PI', 'GTX', 'KRA', 'EROS', 'BLCT', 'TAOP', 'QEP', 'SPPI', 'BLNK',
              'MOGU', 'ADMA']
ohlcv = get_ohlcv(robinhood_client, my_tickers, '5minute', 'week')

In [ ]:
#upcoming earnings 
my_tickers = ['NOK', 'SIRI', 'NOG', 'PCG', 'RWT', 'FCAU', 'CMCSA', 'ICAGY', 'TBIO', 'KTOS']
ohlcv = get_ohlcv(robinhood_client, my_tickers, '5minute', 'week')

==================================================backtesting============================================================

strategy 1: resistance breakout

strategy performance

In [ ]:
pd.set_option('display.max_rows', 500)
ohlcv['T'].index[-1]

In [ ]:
# calculating overall strategy's KPIs
strategy_df = pd.DataFrame()
for ticker in my_tickers:
    strategy_df[ticker] = ohlcv[ticker]["ret"]
strategy_df["ret"] = strategy_df.mean(axis=1)
cagr(strategy_df)
sharpe(strategy_df,0.025)
max_dd(strategy_df)  

In [ ]:
# vizualization of strategy return
%matplotlib inline
(1+strategy_df["ret"]).cumprod().plot()

In [ ]:
#calculating individual stock's KPIs
cagr_dict = {}
sharpe_ratios = {}
max_drawdown = {}
hist_dict = {}
for ticker in my_tickers:
    print("calculating KPIs for ",ticker)      
    cagr_dict[ticker] =  cagr(ohlcv[ticker])
    sharpe_ratios[ticker] =  sharpe(ohlcv[ticker],0.025)
    max_drawdown[ticker] =  max_dd(ohlcv[ticker])
    hist_dict[ticker] = ohlcv[ticker]['hist'].max()
KPI_df = pd.DataFrame([cagr_dict,sharpe_ratios,max_drawdown, hist_dict],index=["Return","Sharpe Ratio","Max Drawdown", "Hist"])      
KPI_df.T

In [ ]:
KPI_df.T[KPI_df.T.Return > 0.01].sort_values(by = ['Return'], ascending = False)

In [ ]:
KPI_df.T[KPI_df.T.Return > 0.01].sort_values(by = ['Return'], ascending = False).iloc[0:20]
my_tickers2 = KPI_df.T[KPI_df.T.Return > 0.01].sort_values(by = ['Return'], ascending = False).iloc[0:20].index.values.tolist()
my_tickers2

In [ ]:
my_tickers2[0:10]